[View in Colaboratory](https://colab.research.google.com/github/leemordechai/Plague/blob/master/InscriptionScraper.ipynb)

In [0]:
import pandas as pd
import urllib3
import requests
from bs4 import BeautifulSoup

!pip install --upgrade -q -gspread
!pip install gspread-dataframe

from google.colab import auth
auth.authenticate_user()

import gspread, gspread_dataframe
from oauth2client.client import GoogleCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [2]:
def get_info(inscription_num): # get all the information from EDB
  s_num = str(inscription_num)
  url = 'http://www.edb.uniba.it/epigraph/' + s_num
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")

  inscription = soup.find('h4').text.strip().split() # get IDs
  id1 = inscription[0]
  try:
    id2 = inscription[1].replace('(','').replace(')','')
  except:
    id2 = ''

  table_text = soup.find('table').text 
  date_loc = table_text.find('Date') # get date
  end_date_loc = table_text[date_loc:].find('\n')
  date = table_text[date_loc+6:date_loc+end_date_loc]
 
  ed_loc = table_text.find('Edition(s):') # get edition
  end_ed = table_text[ed_loc:].replace('\n','',1).find('\n')
  end_ed = table_text[ed_loc:].find('\n\n\n')
  #print(repr(table_text[ed_loc:]))
  #print(end_ed)
  
  ed = table_text[ed_loc+12:ed_loc+end_ed].replace('\n','').replace('\t','')
  ed = ed.replace('\xa0','')

  entry = [id1[3:], id2[2:], date, ed]
  return entry

get_info(40239)

['40239',
 '',
 '521',
 'ICVR II, 4279.1; CIL VI, 9994; ILCV, 1137Other text(s) on the same support:ICVR II, 4279.2Online: EDB13168']

In [11]:
gc = gspread.authorize(GoogleCredentials.get_application_default())
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1kvR_VWLWlENT\
iN7l6FqG7YkhWnrELJyYwJjplC6Yg5c/edit#gid=1660470042').worksheet('EDB-search')

df_search_results = get_as_dataframe(sheet)
pd.to_numeric(df_search_results['ID'], 'integer')
int_list = df_search_results['ID'].iloc[:510].astype('int')
type(int_list)

pandas.core.series.Series

In [158]:
## import the data from EDB

cols = ['ID-EDB', 'ID-TM', 'Date', 'First_edition']
df = pd.DataFrame(columns=cols)
for i in range(len(int_list)):
  try:
    entry = get_info(int_list[i])
  except:
    print('failed: ' + i)
  df.loc[i] = entry
  if i%10 == 0: print(i)
  
df
  
  
  

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500


,ID-EDB,ID-TM,Date,First_edition
0,27280,591604,490-524,"ICVR I, 12; CIL VI, 30115; CLE II, 1338Online:..."
1,9848,299568,490-524,"ICVR I, 34; Cerrito 2011, 361, fig. 4a; Ferrua..."
2,27406,621513,522,"ICVR I, 54; ILCV, 4145 adn."
3,27410,621514,564,"ICVR I, 55; Ferrua 1979b, 10, n. 55; ILCV, 257..."
4,27431,574328,401-525,"ICVR I, 59; CIL VI, 9257; Ferrua 1979b, 10, 57..."
5,27464,621545,487; 503; 510,"ICVR I, 92; ILCV, 3732A adn."
6,35353,700372,500-599,"ICVR I, 93; ICVR V, 15435; Ferrua 1979b, 11, n..."
7,27486,700257,500-599,"ICVR I, 116Online: EDH28291"
8,27488,621566,451-599,"ICVR I, 118; ILCV, 3160 adn."
9,27501,594127,500-599,"ICVR I, 131; CIL VI, 32970; ILCV, 489; Guyon 1..."


In [0]:
from google.colab import files
files.download('EBD.csv')

In [3]:
def get_info_hisp(id_number): # get all the information from Hispania Epigraphica
  s_num = str(id_number)
  url = 'http://eda-bea.es/pub/record_card_2.php?refpage=%2Fpub%2Fsearch_simple.php&field[1][type]=7&field[1][ID]=1&field[2][ID]=2&field[3][ID]=3&field[4][ID]=4&field[5][ID]=5&field[6][ID]=6&field[7][ID]=7&field[12][ID]=12&field[13][ID]=13&rec=' + s_num
  try:
    response = requests.get(url)
  except:
    print ('failed: num ' + s_num)
  soup = BeautifulSoup(response.content, "html.parser")

  table_string = soup.find_all('table')[5].text
  data_start = table_string.find('Datación')
  chrono_start = table_string.find('Cronología')
  date_string = table_string[data_start+8:chrono_start].strip()  
  date_string = date_string.replace("\n",'').replace('\t','')
  date_string = date_string.replace('Año','Año ')
  date_string = date_string.replace('.C.','.C. ')
  
  black_list = ['Siglo I', 'Siglos I', 'siglo I', 'siglos I', 
                'a.C.', 'a. C.', 'Desconocida', 'undefined',
               'más datos resulta imposible datarla',
               'No se aporta', 'Record No.', 'No se aporta',
               'Se desconoce']
  
  for black in black_list:
    if black in date_string: return ''
  
  return date_string

get_info_hisp(1)

'Reinado de Juliano, hacia el 363.'

In [11]:
## import the data from Hispanica
from google.colab import files

cols_he = ['ID-HE', 'Date']
df_he = pd.DataFrame(columns=cols_he)
for i in range(30000, 32594):
  try:
    he_date = get_info_hisp(i)
  except:
    print('failed to get info: ' + str(i))
    he_date = 'failed'
  
  if (len(he_date) > 0):
    df_he.loc[i] = [i, he_date]
  if i%50 == 0: print(i, end=',')
  if i%1000 == 0:
    print()
    df_he.to_csv('HE.csv')
    
df_he.head()

df_he.to_csv('HE.csv')
files.download('HE.csv')

  

30000,
30050,30100,30150,30200,30250,30300,30350,30400,30450,30500,30550,30600,30650,30700,30750,30800,30850,30900,30950,31000,
31050,31100,31150,31200,31250,31300,31350,31400,31450,31500,31550,31600,31650,31700,31750,31800,31850,31900,31950,32000,
32050,32100,32150,32200,32250,32300,32350,32400,32450,32500,32550,

MessageError: ignored

In [0]:
files.download('HE.csv')

In [13]:
1231*25

30775